In [2]:
import os # to listdir
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix , accuracy_score
from PIL import Image, ImageFilter

In [117]:
im_width = int(480/8)
im_height = int(640/8)

#im = Image.open('Landing/training/Flat/F2.jpg').convert('L')
im = Image.open('Landing/training/Rough/IMG_2809.jpg').convert('L')
#im.show()
#print(im.width, im.height) #-> 480 640
#print(np.asarray(im)); print(np.asarray(im).shape); print('\n') #-> (640, 480)

#edges_image = im.filter(ImageFilter.FIND_EDGES)
#edges_image.show()

im = im.resize((im_height, im_width))
#im.show()
#print(im.width, im.height) #-> 80 60

im_array = np.asarray(im).astype(float) #-> (60, 80)
#print(im_array); print(im_array.shape);
#print(f'{np.min(np.abs(im_array))}'); print(f'{np.max(np.abs(im_array))}'); print('\n')

# standardization
im_array = (im_array - np.min(im_array)) * 255 / (np.max(im_array) - np.min(im_array))
#print(im_array); print(im_array.shape);
#print(f'{np.min(np.abs(im_array))}'); print(f'{np.max(np.abs(im_array))}'); print('\n')

#x = im_array.reshape(1, -1) #-> (1, 4800)
#y = np.array([0]) #-> (1,)
#print(x); print(x.shape);
#print(f'{np.min(np.abs(x))}'); print(f'{np.max(np.abs(x))}'); print('\n')

# finding the edges
im = Image.fromarray(im_array).convert('L')
#im.show()
edges_image = im.filter(ImageFilter.FIND_EDGES) #-> PIL.Image.Image
#edges_image.show()

edges_array = np.asarray(edges_image).astype(float) #-> numpy.ndarray
#print(edges_array); print(edges_array.shape) #-> (60, 80)
#print(f'{np.min(np.abs(edges_array))}'); print(f'{np.max(np.abs(edges_array))}'); print('\n')

# excluding the peripheral pixels
edges_array_scaled = edges_array.copy()[1:im_width-1,1:im_height-1] #-> (58, 78)
#print(edges_array_scaled); print(edges_array_scaled.shape)
#print(f'{np.min(np.abs(edges_array_scaled))}'); print(f'{np.max(np.abs(edges_array_scaled))}'); print('\n')

# setting a threshold for edges
edge_thresh = 64
edges_array_scaled[edges_array_scaled < edge_thresh,] = 0
print(edges_array_scaled); print(edges_array_scaled.shape)
print(f'{np.min(np.abs(edges_array_scaled))}'); print(f'{np.max(np.abs(edges_array_scaled))}'); print('\n')

# engineer features
edges_v = np.sum(edges_array_scaled, axis=0).astype(float)
print(edges_v); print(edges_v.shape);
peak_v = np.max(np.abs(edges_v))
print(peak_v); print('\n')

edges_h = np.sum(edges_array_scaled, axis=1).astype(float)
print(edges_h); print(edges_h.shape);
peak_h = np.max(np.abs(edges_h))
print(peak_h); print('\n')

# high pass filter
M = 5

for i in range(len(edges_v)-M):
    edges_v[i] = edges_v[i] - np.mean(edges_v[i:i+M])
edges_v[-M:] = 0
print(edges_v); print(edges_v.shape);
peak_v = np.max(np.abs(edges_v))
print(np.min(np.abs(edges_v))); print(peak_v); print('\n')
    
for i in range(len(edges_h)-M):
    edges_h[i] = edges_h[i] - np.mean(edges_h[i:i+M])
edges_h[-M:] = 0
print(edges_h); print(edges_h.shape);
peak_h = np.max(np.abs(edges_h))
print(np.min(np.abs(edges_h))); print(peak_h); print('\n')

peak_thresh = 0.5
noise_v = edges_v.copy()[np.abs(edges_v) < peak_thresh*peak_v]
print(noise_v); print(noise_v.shape); print(np.max(np.abs(noise_v))); print('\n')
noise_h = edges_h.copy()[np.abs(edges_h) < peak_thresh*peak_h]
print(noise_h); print(noise_h.shape); print(np.max(np.abs(noise_h))); print('\n')

# finding the standard deviation
sd_v = np.sqrt(np.var(noise_v))
print(sd_v); print('\n')
sd_h = np.sqrt(np.var(noise_h))
print(sd_h); print('\n')

x = np.array([peak_v/sd_v, peak_h/sd_h]).reshape(1,-1)
print(x); print(x.shape)
"""
"""

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(58, 78)
0.0
255.0


[    0.    98.   158.     0.    80.     0.     0.     0.     0.  5393.
 12419.     0.     0.   526.     0. 10750.  9059.     0.   344.     0.
     0.     0.     0.     0.    67.    89.  3305. 14479.     0.     0.
   233.   133.     0.  3518. 14790.    66.    74.   129.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.   224.     0.  9908. 14790.     0.     0.   144.   218.
     0.   205.     0.     0. 13869.  1131.     0.     0.     0.     0.
     0.    89.     0.     0.     0.     0.     0.     0.]
(78,)
14790.0


[2357. 2238. 2231. 2251. 2284. 2336. 2296. 2404. 2323. 2320. 2571. 2496.
 2143. 2336. 2202. 2164. 2056. 1974. 1868. 2024. 2028. 2082. 2106. 2123.
 2241. 2019. 2006. 2072. 2088. 1943. 1954. 2010. 1874. 2078. 1856. 2055.
 2118. 1862. 1890. 1888. 1830. 18

'\n'